In [15]:
from tqdm import tqdm
import network
import utils
import os
import random
import argparse
import numpy as np
import json

from torch.utils import data
from datasets import Cityscapes
from utils import ext_transforms as et
from metrics import StreamSegMetrics

import torch
import torch.nn as nn
import wandb

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

from torchvision import transforms
# from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import DataLoader, Dataset
from glob import glob
from collections import namedtuple
from dataloaders import DataProcessor, KITTI360Dataset, DatasetLoader
from weaklabelgenerator import labelgenerator
import network


In [20]:
datapath = 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/'
datapath = datapath.replace('\\', '/')
print(datapath)
# datapath = 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/'

datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/


In [21]:
files = os.listdir(datapath)
print(len(files))

11518


2000


['0000010263.png',
 '0000005307.png',
 '0000003782.png',
 '0000006304.png',
 '0000009918.png']

In [14]:
# Sample every 5th image from the sorted files. Like image 0, 4, 9, 14....
seq_image_paths = sorted(files)
seq_image_paths = seq_image_paths[::5]
print(len(seq_image_paths))
print(seq_image_paths[0:5])

2304
['0000000000.png', '0000000005.png', '0000000010.png', '0000000015.png', '0000000020.png']


In [26]:
latest_file = 'kitti_360_00_full_val.json'
with open(latest_file, 'r') as f:
    data = json.load(f)


print(len(data))

11518


In [29]:
count_train = 0
count_val = 0
new_images = []
for entry in data:
    if not entry['val_set']:
        count_train += 1
        new_images.append(entry['image'])
    else:
        count_val += 1


print(count_train, count_val)
print(len(new_images))



9105 2413
9105


In [32]:
# Generate entropy, confidence, class counts from weaklabelgenerator
from weaklabelgenerator_v1 import labelgenerator

model_name = 'deeplabv3plus_resnet101'
ckpt = "checkpoints/best_deeplabv3plus_resnet101_cityscapes_os16.pth"
teacher_model = network.modeling.__dict__[model_name](num_classes=19, output_stride=16)
model = network.modeling.__dict__[model_name](num_classes=19, output_stride=16)

# Starting weaklabeling
filtered_samples, train_labelgen = labelgenerator(imagefilepaths=new_images, model=model, ckpt=ckpt)

Device: cuda
Resume model from checkpoints/best_deeplabv3plus_resnet101_cityscapes_os16.pth


100%|██████████| 9105/9105 [17:37<00:00,  8.61it/s]


Overall average entropy for all images: 0.1508
Overall average confidence for all images: 0.9492
Saved image and label paths to kitti_360_00_full_filtered.json


In [33]:

with open('kitti_360_00_full_filtered.json', 'r') as f:
    filtered_data = json.load(f)

In [34]:
len(filtered_data)

9105

In [35]:

image_paths_2k = random.sample(filtered_data, 2000)

# seq_image_paths = sorted(files)
# seq_image_paths = seq_image_paths[::5]



In [39]:
print(len(image_paths_2k))
print(image_paths_2k[0:5])

json_name ='kitti_360_00_2k_random.json'
with open(json_name, 'w') as f:
    json.dump(image_paths_2k, f, indent=4)

2000
[{'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000010550.png', 'entropy': 0.05401115491986275, 'confidence': 0.9826192259788513, 'num_classes': 10}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000104.png', 'entropy': 0.14523471891880035, 'confidence': 0.9511430859565735, 'num_classes': 13}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000011190.png', 'entropy': 0.19926035404205322, 'confidence': 0.9372466206550598, 'num_classes': 12}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000004275.png', 'entropy': 0.21424955129623413, 'confidence': 0.9234609603881836, 'num_classes': 12}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000006287.png', 'entropy': 0.12285968661308289, 'confidence': 0.9571988582611084, 'num_clas

In [40]:
filtered_data[0:5]

[{'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000000.png',
  'entropy': 0.13571812212467194,
  'confidence': 0.9538878798484802,
  'num_classes': 10},
 {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000001.png',
  'entropy': 0.13995672762393951,
  'confidence': 0.9519590139389038,
  'num_classes': 9},
 {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000002.png',
  'entropy': 0.14549696445465088,
  'confidence': 0.9502736330032349,
  'num_classes': 9},
 {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000003.png',
  'entropy': 0.1455216258764267,
  'confidence': 0.9502925872802734,
  'num_classes': 9},
 {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000004.png',
  'entropy': 0.15189900994300842,
  'confidence': 0.9489

In [41]:
seq_image_paths = filtered_data[::5]
print(len(seq_image_paths))
print(seq_image_paths[0:5])

1821
[{'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000000.png', 'entropy': 0.13571812212467194, 'confidence': 0.9538878798484802, 'num_classes': 10}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000005.png', 'entropy': 0.15387742221355438, 'confidence': 0.9474475383758545, 'num_classes': 9}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000010.png', 'entropy': 0.16455146670341492, 'confidence': 0.9431623816490173, 'num_classes': 10}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000015.png', 'entropy': 0.15756496787071228, 'confidence': 0.9463582634925842, 'num_classes': 11}, {'image_path': 'datasets/data/KITTI-360/data_2d_raw/2013_05_28_drive_0000_sync/image_00/data_rect/0000000020.png', 'entropy': 0.1493283361196518, 'confidence': 0.9486227035522461, 'num_classe

In [42]:
seq_json_name = 'kitti_360_00_2k_seq.json'
with open(seq_json_name, 'w') as f:
    json.dump(seq_image_paths, f, indent=4)

In [73]:
import json
import numpy as np

def filter_rich_data(json_file_path, entropy_threshold=0.20, confidence_threshold=0.85, min_class_count=1):
    # Load the JSON file
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Filter entries based on confidence and class count
    filtered_data = [
        entry for entry in data 
        if float(entry["entropy"]) >= entropy_threshold 
        # if float(entry['confidence']) >= confidence_threshold 
        and int(entry['num_classes']) >= min_class_count
    ]

    # Sort the filtered data by entropy (descending) to prioritize rich data representation
    filtered_data.sort(key=lambda x: float(x['entropy']), reverse=True)

    print(f"Total entries: {len(data)}")
    print(f"Filtered entries: {len(filtered_data)}")

    return filtered_data

# Usage
json_file_path = 'kitti_360_00_full_detailed.json'
filtered_entries = filter_rich_data(json_file_path, entropy_threshold=0.19, confidence_threshold=0.8, min_class_count=1)

# print(len(filtered_entries))
# Optionally, save the filtered entries to a new JSON file
if filtered_entries:
    with open('kitti_360_00_filtered.json', 'w') as file:
        json.dump(filtered_entries, file, indent=4)
    # print(f"Filtered data saved to 'outputs/weaklabels/KITTI-360/filtered_data.json'")
else:
    print("No entries matched the filter criteria.")

Total entries: 9105
Filtered entries: 2161


In [74]:
with open('kitti_360_00_2k_random.json', 'r') as f:
    data = json.load(f)

for entry in data:
    entry['val_set'] = False

with open('kitti_360_00_2k_random.json', 'w') as f:
    json.dump(data, f, indent=4)

with open('kitti_360_00_2k_seq.json', 'r') as f:
    data = json.load(f)

for entry in data:
    entry['val_set'] = False

with open('kitti_360_00_2k_seq.json', 'w') as f:
    json.dump(data, f, indent=4)

with open('kitti_360_00_filtered.json', 'r') as f:
    data = json.load(f)

for entry in data:
    entry['val_set'] = False

with open('kitti_360_00_filtered.json', 'w') as f:
    json.dump(data, f, indent=4)

